# Estimating the Carbon Footprint of the Neural Network Training
In this tutorial we will demonstrate how to use the **codecarbon** Python package to **estimate** the carbon footprint produced by training a neural network.

## Import dependencies
We will build a Neural Network using the Keras Sequential API.

In [1]:
from codecarbon import EmissionsTracker

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import pad_sequences

2023-01-21 00:10:51.287837: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Setting the hyper parameters

In [2]:
max_features = 10000 # vocabulary size
maxlen = 256 # The length of every input sequence

embedding_size = 64  # the dimension of the embeddings
dropout = 0.1
filters = 32  # num of 1D convolution filters (output num channels)
kernel_size = 5  # 1D convolution kernel width
pool_size = 4
lstm_output_size = 16  # number of LSTM units
batch_size = 32

epochs = 100  # number of training epochs
test_batch_size = 32  # batch size for testing

## Loading the data
We will load the IMDb sentiment analysis dataset directly from the Keras dataset module

In [3]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 256)
x_test shape: (25000, 256)


## Building the model
We will build a simple model typically used for sentiment analysis.

In [4]:
def make_model(
    metrics: list,
    vocab_size: int,
    embedding_dim: int,
    dropout: float,
    filters: int,
    kernel_size: int,
    pool_size: int,
    lstm_output_size: int,
) -> Sequential:
    """Builds a simple Keras model.

    Args:
        metrics (list): list of metrics to report

        vocab_size (int): nunmber of tokens in the vocabulary.
        It determines the embedding look-up yable size.

        embedding_dim (int): the dimensionbality of the
        embedding vectors.

        dropout (float): dropout rate after the embedding layer.

        filters (int): number of output channels of the 1D convolution.

        kernel_size (int): the kernel size (width) of the 1D convolution.

        pool_size (int): the size (width) of the 1D max pooling layer.

        lstm_output_size (int): the dimensionality of the output vectors
        from the final bi-LSTM layer.

    Returns:
        Sequential: compiled Keras sequential model
    """
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=maxlen),
        Dropout(dropout),
        Conv1D(filters, kernel_size, padding='valid', activation='relu'),
        MaxPooling1D(pool_size=pool_size),
        Bidirectional(LSTM(lstm_output_size), merge_mode='ave'),
        Dense(1),
        Activation('sigmoid'),
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=metrics
    )

    return model

## Training the model
We write a simple function to train the model. As we don't use cross-validation, we will use the training data as validation.

In [5]:
def train_model():
    model = make_model(
        metrics=["accuracy"],
        vocab_size=max_features,
        embedding_dim=embedding_size,
        dropout=dropout,
        filters=filters,
        kernel_size=kernel_size,
        pool_size=pool_size,
        lstm_output_size=lstm_output_size,
    )

    h = model.fit(
        x_train,
        y_train,
        validation_data=(x_test, y_test),
        batch_size=batch_size,
        epochs=epochs,
    )

## Estimating the CO2 emissions
This is where we start tracking the CO2 emissions. It is as simple as timing the code.

In [ ]:
tracker = EmissionsTracker(project_name="imbd_sentiment_classification")
tracker.start()
train_model()
tracker.stop()